In [ ]:
#!/usr/bin/env python3
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import matplotlib

from wilson2 import draw_sampling, Wilson, trace_estimator

import sys
sys.path.append('/home/carlo/workspace/networkqit')
import networkqit as nq

In [ ]:
def factorial(x):
    if x==0 or x==1:
        return 1
    else:
        return x*factorial(x-1)

In [ ]:
import networkx as nx
import autograd.numpy as np
from autograd import grad
import scipy
import matplotlib.pyplot as plt
from functools import partial

#G = nx.erdos_renyi_graph(64,0.8)
G = nx.ring_of_cliques(5,10)
A  = nx.to_numpy_matrix(G)
D = np.asarray(np.diagflat(A.sum(0)))
invD  = np.asarray(np.diagflat(1/A.sum(0)))
n=nx.number_of_nodes(G)
I = np.eye(n)

L = nx.laplacian_matrix(G).toarray()
lambda_max = np.linalg.eigvalsh(L).max()

order = 3
beta_range = np.logspace(-3,3,100)

lambdai = np.linalg.eigvalsh(L)

def r(beta, L, order=10):
    return np.sum([( (-1.0)**k/factorial(k) + (-1.0)**(k+1)/k)*(beta**k)*np.trace(np.linalg.matrix_power(L,k)) for k in range(3,order)])

def Z(beta, L):
    return np.trace(scipy.linalg.expm(-beta*L))

def logR(beta,L):
    #return np.trace(scipy.linalg.logm(np.eye(n) + beta*L))
    return np.linalg.slogdet( np.eye(n) + beta*L )[1]

def logChi(beta,L):
    q= 1.0 / beta
    #return np.linalg.slogdet(q * I + L)[1] - n * np.log(q)
    return np.trace(scipy.linalg.logm(q * I + L)) - n * np.log(q)

def E(beta,L):
    return np.trace(L@scipy.linalg.expm(-beta*L))/Z(beta,L)

rbeta = np.asarray([r(beta,L) for beta in beta_range])
Zbeta = np.asarray([Z(beta,L) for beta in beta_range])
logChibeta = np.asarray([logChi(beta,L) for beta in beta_range])
Ebeta = np.asarray([E(beta,L) for beta in beta_range])

In [ ]:
plt.figure(figsize=(7.5*1.61,7.5))
plt.semilogx(beta_range, Zbeta, label='$Z=\\mathrm{Tr}[e^{-\\beta \\mathbf{L}}]$')
plt.semilogx(beta_range, n-logChibeta + rbeta, label='$Z=n-\\log \\chi\\left(\\beta^{-1} \\right) + n\\log\\beta + r(\\beta)$')
plt.grid(which='both',linestyle='dotted',alpha=0.8)
plt.axvline(x=1/lambdai.max(), ymin=0, ymax=n, color='black', linestyle='dashed')
plt.legend(fontsize=16,loc='best')
plt.title('Partition function', fontname='Helvetica',fontsize=16)
plt.text(x=1/lambdai.max()+0.01,y=2,s='$\\lambda_{max}^{-1}$',fontsize=16)
plt.xlabel('$\\beta$', fontsize=16, fontname='Helvetica')
plt.xticks(fontsize=14, fontname='Helvetica')
plt.yticks(fontsize=14, fontname='Helvetica')
plt.ylim([0,n])
#plt.savefig('z_vs_approx.png', dpi=200, bbox_inches='tight')

In [ ]:
#Eapprox3 = np.asarray([-(1/beta)*np.sum(beta/(1+lambdai))/(n-logChi[i]) for i,beta in enumerate(beta_range)])
Eapproxbeta =  np.asarray([1/Z(beta,L)*np.trace(np.linalg.inv(I+beta*L)@L - I/beta) for beta in beta_range])
plt.figure(figsize=(7.5*1.61,7.5))
plt.semilogx(beta_range, Ebeta, label='$E=\\mathrm{Tr}[\\rho \\mathbf{L}]$')
plt.semilogx(beta_range, Eapproxbeta, label='$E=\\mathrm{Tr}[\\rho \\mathbf{L}]$')
plt.grid(which='both',linestyle='dotted',alpha=0.8)
plt.axvline(x=1/lambdai.max(), ymin=0, ymax=n, color='black', linestyle='dashed')
plt.legend(fontsize=16,loc='best')
plt.title('Energy', fontname='Helvetica',fontsize=16)
plt.text(x=1/lambdai.max()+1E-2,y=0.1,s='$\\lambda_{max}^{-1}$',fontsize=16)
plt.xlabel('$\\beta$', fontsize=16, fontname='Helvetica')
plt.xticks(fontsize=14, fontname='Helvetica')
plt.yticks(fontsize=14, fontname='Helvetica')
#plt.xlim([1E-3,1E0])
#plt.ylim([Ebeta.min(),Ebeta.max()])